In [ ]:
opts = {}

In [ ]:
opts["size"] = 12

In [ ]:
opts["hidden_dim"] = 128
opts["depth"] = 3

In [ ]:
opts["rep_dim"] = 4

In [ ]:
layer_info = [[opts["size"]**2] + [opts["hidden_dim"]]*opts["depth"] + [opts["rep_dim"]]]

In [ ]:
layer_info